<a href="https://colab.research.google.com/github/HasunJung/llm_langchain/blob/main/03_output_parsers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Output Parsers 모듈은 언어 모델에서 얻은 출력을 분석해 애플리케이션에서 사용하기 쉬운 형태로 변환하는 기능을 제공합니다.
출력 문자열을 정형화하거나 특정 정보를 추출하는데 사용합니다.

이 모듈을 사용하면 출력을 구조화된 데이터로 쉽게 처리할 수 있습니다.

그럼 테디노트 출력 파서 예제를  gemini로 변경하여 따라해봅니다.

# INSTALL

In [ ]:
# 설치
!pip install -q langchain langchain-google-genai

import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.5/664.5 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.4 MB/s eta 0:00:00


Output parsers는 LLM의 출력을 가져와 보다 적합한 형식으로 변환하는 역할을 담당합니다. 이는 LLM을 사용하여 모든 형태의 구조화된 데이터를 생성할 때 매우 유용합니다.

다양한 유형의 출력 파서 컬렉션을 보유하는 것 외에도 LangChain OutputParsers의 한 가지 눈에 띄는 이점은 그 중 다수가 스트리밍을 지원한다는 것입니다.

https://python.langchain.com/docs/modules/model_io/output_parsers/

In [ ]:
# 정의한 모델과 템플릿은 아래 섹션에서 사용합니다.
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# ChatGoogleGenerativeAI 언어 모델을 "gemini-pro" 모델로 초기화합니다.
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)

invoke(), stream(), batch() 인터페이스를 사용하여 입출력을 처리합니다.

In [ ]:
# 언어 모델을 사용하여 프롬프트를 전달하여 결과를 생성합니다.
result = llm.invoke("오늘 점심 메뉴 추천해줘")

print(result.content)  # 생성된 결과의 내용을 출력합니다.

**가벼운 점심**

* **그릴드 치킨 샐러드:** 혼합 채소, 그릴드 치킨, 크루통, 파마산 치즈, 드레싱
* **투나 샌드위치:** 통밀 빵, 투나, 마요네즈, 양파, 셀러리
* **요거트 파르페:** 요거트, 과일, 그래놀라

**풍성한 점심**

* **파스타 카르보나라:** 파스타, 베이컨, 달걀, 파마산 치즈
* **피자:** 원하는 토핑과 함께
* **버거:** 빵, 패티, 치즈, 토핑

**건강한 점심**

* **퀴노아 볼:** 퀴노아, 채소, 콩, 허브
* **렌즈콩 수프:** 렌즈콩, 채소, 향신료
* **그릴드 연어와 구운 야채:** 연어 필레, 구운 브로콜리, 당근, 감자

**특별한 점심**

* **수시:** 다양한 생선, 해산물, 밥
* **타코:** 옥수수 또는 밀 토르티야, 고기 또는 채소, 토핑
* **파드 타이:** 쌀국수, 닭고기 또는 새우, 채소, 땅콩 소스

**디저트**

* **과일 샐러드:** 신선한 과일, 요거트 또는 꿀
* **브라우니:** 초콜릿 브라우니, 아이스크림 또는 휘핑 크림
* **쿠키:** 초콜릿 칩 쿠키, 오트밀 쿠키, 슈가 쿠키


# PydanticOutputParser

사용자는 임의의 Pydantic 모델을 지정하고 해당 스키마를 준수하는 출력에 대해 LLM을 쿼리할 수 있습니다.

In [ ]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [ ]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 format을 추가
prompt = prompt.partial(format=parser.get_format_instructions())

In [ ]:
chain = prompt | llm


In [ ]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
print(response.content)

```json
{"person": "\uc774\uc5f0\uc218", "email": "chulsoo.kim@bikecorporation.me", "subject": "\"ZENESIS\" \uc9c0\uc9c0\ub97c \uc720\ud29c \ud56d\uc0c1 \ubc0f \ubc18\ud658 \uc81c\ubaa9", "summary": "\uc774\uc5f0\uc218\uc758 \ubc18\ud658\uc744 \uc54c\ub824\uc8fc\uc138\uc694. \uc774\uc5f0\uc218\uc758 \ubc18\ud658\uc744 \uc9c0\uc9c0\ub97c \uc720\ud29c\ud55c \uc720\ud29c \uc81c\ubaa9\uc744 \uc81c\uc57d\ud558\uc2ed\uc2dc\uc624.", "date": null}
```


In [ ]:
#parser.parse(response.content)

In [ ]:
# 출력 파서를 추가하여 전체 체인을 재구성합니다.
chain = prompt | llm | parser


In [ ]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
response

# CommaSeparatedListOutputParser

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()
# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    # 주제에 대한 다섯 가지를 나열하라는 템플릿
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],  # 입력 변수로 'subject' 사용
    # 부분 변수로 형식 지침 사용
    partial_variables={"format_instructions": format_instructions},
)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | llm | output_parser

In [ ]:
chain.invoke(
    {"subject": "용인시 관광명소"}
)  # "대한민국 관광명소"에 대한 체인 호출 실행

['에버랜드', '용인대장금파크', '남사당', '포곡선수촌', '용인시립박물관']

In [ ]:
# 용인시 관광명소에 대한 스트림을 순회합니다.
for s in chain.stream({"subject": "용인시 관광명소"}):
    print(s)  # 스트림의 내용을 출력합니다.

['에버랜드']
['용인대장금파크']
['남사당']
['포곡선수촌']
['용인시립박물관']


# StructuredOutputParser

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 출처, 웹사이트 이여야 합니다.",
    ),
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [ ]:
chain = prompt | llm | output_parser  # 프롬프트, 모델, 출력 파서를 연결

In [ ]:
# 대한민국의 수도가 무엇인지 질문합니다.
chain.invoke({"question": "대한민국의 수도는 어디인가요?"})

{'answer': '서울',
 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8%ED%8A%B9'}

In [ ]:
for s in chain.stream({"question": "세종대왕의 업적은 무엇인가요?"}):
    # 스트리밍 출력
    display(s)

{'answer': '세종대왕의 주요 업적은 다음과 같습니다.\n\n* **한글 창제:** 세종대왕은 훈민정음(한글)을 창제하여 조선 백성들이 쉽게 글을 읽고 쓸 수 있도록 했습니다.\n* **과학 기술 발전:** 세종대왕은 천문학, 지리학, 의학 등 다양한 과학 기술 분야를 지원하여 조선의 과학 기술 발전에 기여했습니다.\n* **법전 정비:** 세종대왕은 국민의 권리를 보호하고 사회 질서를 유지하기 위해 법전을 정비했습니다.\n* **교육 제도 개선:** 세종대왕은 국립대학인 성균관을 확장하고 사학을 지원하여 교육 제도를 개선했습니다.\n* **문화적 번영:** 세종대왕은 예술과 문학을 장려하여 조선의 문화적 번영에 기여했습니다.',
 'source': 'https://www.doopedia.co.kr/doopedia/master/master.do?_method=view&MAS_IDX=10101300072643'}

# JsonOutputParser

In [ ]:
from typing import List

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [ ]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="Concise description about topic")
    hashtags: str = Field(description="Some keywords in hashtag format")

In [ ]:
# 질의 작성
query = "온난화에 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)

prompt = PromptTemplate(
    # 사용자 쿼리에 답하십시오.
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # 입력 변수 설정
    # 부분 변수에 형식 지시사항 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser  # 체인을 구성합니다.

chain.invoke({"query": query})  # 체인을 호출하여 쿼리 실행

{'description': '지구 온난화는 인간 활동으로 인해 대기 중 이산화탄소와 기타 온실 가스의 농도가 증가하여 지구의 평균 기온이 상승하는 현상입니다.',
 'hashtags': '#지구온난화 #기후변화 #온실가스'}

Without Pydantic
Pydantic 없이도 이 기능을 사용할 수 있습니다. 이 경우 JSON을 반환하도록 요청하지만, 스키마가 어떻게 되어야 하는지에 대한 구체적인 정보는 제공하지 않습니다.

JsonOutputParser 클래스의 인스턴스를 parser 변수에 할당합니다.
PromptTemplate 클래스를 사용하여 사용자 쿼리에 대한 프롬프트를 생성합니다. 이때, template 매개변수에는 사용자 쿼리를 처리할 템플릿 문자열을, input_variables에는 템플릿에서 사용할 변수 목록을, partial_variables에는 parser에서 제공하는 형식 지침을 포함합니다.
prompt, model, parser를 연결하여 chain을 구성합니다.
chain.invoke 메소드를 호출하여 {"query": query} 매개변수를 전달함으로써, 사용자 쿼리에 대한 처리를 시작합니다.

In [ ]:
# 질의 작성
query = "온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

parser = JsonOutputParser()  # JSON 출력 파서 초기화

prompt = PromptTemplate(
    # 사용자 쿼리에 답변하는 템플릿
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # 입력 변수로 'query' 사용
    # 부분 변수로 포맷 지시사항 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser  # 프롬프트, 모델, 파서를 연결하는 체인 생성

chain.invoke({"query": query})  # 체인을 호출하여 농담 쿼리 처리

{'description': '지구 온난화는 인간 활동으로 인해 대기 중 온실 가스 농도가 증가하여 지구의 평균 기온이 상승하는 현상입니다. 주요 온실 가스는 이산화탄소, 메탄, 아산화질소 등이며, 이러한 가스는 태양으로부터 들어오는 열을 가두어 지구의 온도를 상승시킵니다.',
 'hashtags': ['#지구온난화', '#온실가스', '#기후변화', '#환경오염', '#지속가능성']}

# SimpleJsonOutputParser

In [ ]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | llm | json_parser

In [ ]:
list(json_chain.stream({"question": "Who invented the microscope?"}))

[{'answer': 'Zacharias Janssen'}]